In [ ]:
# 计算语料库的平均长度

file_path = 'data/wiki1m_for_simcse.txt'

total_length = 0
total_sentences = 0

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        words = line.strip().split()
        total_length += len(words)
        total_sentences += 1

average_length = total_length / total_sentences
print(average_length)


In [ ]:
# 分词测试
import spacy

# 加载英语语言模型
nlp = spacy.load("en_core_web_sm")

# 分词示例
text = "South Australia (SA)  has a unique position in Australia's history as, unlike the other states which were founded as colonies, South Australia began as a self governing province Many were attracted to this and Adelaide and SA developed as an independent and free thinking state."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

# # 关键词提取示例
# print("\n关键词提取:")
# keywords = [token.text for token in doc if not token.is_stop and token.is_alpha]

# # 选择关键词列表中的第一个词作为最关键的词
# most_important_word = keywords[0] if keywords else None
# print(most_important_word)




In [ ]:
# es
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts="http://59.77.134.205:9200")

index_name = "wikidata-latest"

query = {"query": {"match": {"content": "China"}}}
result = es.search(index=index_name, body=query)
print(result)



## spacy记录

不同模型有不同精度

#### [en_core_web_sm](https://spacy.io/models/en#en_core_web_sm)

支持的实体类型

<img src="https://lyu-oss.oss-cn-beijing.aliyuncs.com/img-bed/image-20231227161004504.png" alt="image-20231227161004504" style="zoom:50%;" />

* CARDINAL -- 数字 【'九百多', '8000', '八百'，'1111.01'】
* DATE -- 大粒度时间，时间段 【 '今年', '明天', '今天', '国庆期间', '3天', '10天'， '三年前'】
* **EVENT -- 事件 【'伦敦奥运会', '世界杯','第14届中国国际工业博览会', '深圳市五届人大二次会议'】**
* **FAC -- 小地点 【'轻轨1号线锡北运河站', '万达广场', '乐购超市'，'永盛大酒店', '110岗亭'】**
* **GPE -- 地点 【'美国', '加拿大', '北京', '中国',】**
* **LANGUAGE -- 语言 【'英语', '汉语', '上海话', '中文'】**
* **LAW -- 规章制度 【'青少年犯罪法', '阿鲁巴决议', '反托拉斯法'】**
* **LOC -- 大地点 【'欧洲人', '欧洲', '亚洲', '天山山脉', '巴尔斯卡乌尼河''月球', '火星'】**
* MONEY -- 货币 【'￥8000', '9200', '60元', '3000美金'】部分省略货币单位也能识别
* **NORP -- 人物、地点 【'德国', '中国', '中国人','韩版', '日媒', '日本', '扬州'】**
* ORDINAL -- 顺序 【'首', '第六', '第二', '第一','第几'】
* **ORG -- 组织 【'杭州江干区公安分局'，'LG', '三星', '苹果'，'中国移动', '央行'，'中央社'， '外交部'】**
* PERCENT -- 比率 【'0.2%', '0.2%', '48%'】
* **PERSON -- 人物 【'栾丽娜', '斯蒂芬·弗雷斯', '栗元广', '小雨', '小银狐'】**
* **PRODUCT -- 产品 【'Android', 'iOS'， '金龙鱼', 'UCWeb的浏览器'】**
* QUANTITY -- 量级 【'87,000', '46.522吨', '48248.8千克', '0.23点'，'5.3级', '4200公里' 】
* TIME -- 时间 【'十分钟', '下午', '七点', '今晚'】
* **WORK_OF_ART -- 艺术品 【'刺客信条', '富春山居图', '有一说一'，'探索•发现', '定窑考工记'】**

事件、地点、语言、规章制度、人物、组织、产品、艺术品

In [10]:
#分词统计
# 统计语料库中各种类型的实体数量

# 事件、地点、语言、规章制度、人物、组织、产品、艺术品
import spacy
from tqdm import tqdm
from collections import Counter

type_list = ['EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'NORP', 'ORG', 'PERSON', 'PRODUCT', 'WORK_OF_ART']

nlp = spacy.load("en_core_web_sm")

file_path = 'data/wiki1m_for_simcse.txt'

type_counts = Counter()

with open(file_path, 'r', encoding='utf-8') as file:
    for index, line in tqdm(enumerate(file)):
        words = line.strip().split()
        text = ' '.join(words)
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ in type_list:
                type_counts[ent.label_] += 1
    

for type_name, count in type_counts.items():
    print(f"{type_name}: {count}")



# GPE: 373869
# NORP: 152530
# ORG: 532072
# WORK_OF_ART: 80636
# FAC: 37106
# PERSON: 505931
# LOC: 56057
# PRODUCT: 22275
# LAW: 6742
# EVENT: 29668
# LANGUAGE: 8100
# sum: 1,763,006
    
# 事件:P31: Q1656682
# 地点:P31: Q2221906
# 语言:P31: Q34770
# 规章制度:P31: Q22097341
# 人物:P31: Q5
# 组织:P31: Q43229
# 产品:P31: Q2424752
# 艺术品:P31: Q838948

1000000it [1:23:15, 200.19it/s]

GPE: 373869
NORP: 152530
ORG: 532072
WORK_OF_ART: 80636
FAC: 37106
PERSON: 505931
LOC: 56057
PRODUCT: 22275
LAW: 6742
EVENT: 29668
LANGUAGE: 8100
